In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

In [30]:
# signal magnitude vector
def smv(accX:float, accY:float, accZ:float)->float:
    return math.sqrt(pow(accX,2)+pow(accY,2)+pow(accZ,2))

# slope of the line between max_smv point and min_smv point
def slope(max_smv_x:float, max_smv_y:float, max_smv_z:float, 
         min_smv_x:float, min_smv_y:float, min_smv_z:float)->float:
    return math.sqrt(pow(max_smv_x - min_smv_x, 2) + pow(max_smv_y - min_smv_y, 2) + pow(max_smv_z - min_smv_z, 2))

In [32]:
class Window:
    def __init__(self, raw_df, label):
        self.raw_df = raw_df
        self.label = label
        dict_feature = Window.extract(raw_df)
        
        self.smv_list = dict_feature['smv_list']
        self.mean_ax = dict_feature['mean_ax']
        self.mean_ay = dict_feature['mean_ay']
        self.mean_az = dict_feature['mean_az']
        self.mean_gx = dict_feature['mean_gx']
        self.mean_gy = dict_feature['mean_gy']
        self.mean_gz = dict_feature['mean_gz']
        self.mean_smv = dict_feature['mean_smv']
        self.std_ax = dict_feature['std_ax']
        self.std_ay = dict_feature['std_ay']
        self.std_az = dict_feature['std_az']
        self.std_gx = dict_feature['std_gx']
        self.std_gy = dict_feature['std_gy']
        self.std_gz = dict_feature['std_gz']
        self.std_smv = dict_feature['std_smv']
        self.std_motionless = dict_feature['std_mless']
        self.max_smv = dict_feature['max_smv']
        self.min_smv = dict_feature['min_smv']
        self.slope = dict_feature['slope']
        self.duration = dict_feature['duration']

    @staticmethod
    def extract(df:pd.DataFrame):
        smv_result = []
        for line in range(0, df.shape[0]):
            acc_x = df['acc_x'][line]
            acc_y = df['acc_y'][line]
            acc_z = df['acc_z'][line]
            smv_result.append(smv(acc_x, acc_y, acc_z))
            
#             gryo_x = df['gryo_x'][line]
#             gryo_y = df['gryo_y'][line]
#             gryo_z = df['gryo_z'][line]
                
        smv_list = np.asarray(smv_result)
        min_idx = np.argmin(smv_list)
        max_idx = np.argmax(smv_list)
        
        mean_ax = np.mean(df['acc_x'])
        mean_ay = np.mean(df['acc_y'])
        mean_az = np.mean(df['acc_z'])
        mean_gx = np.mean(df['gyro_x'])
        mean_gy = np.mean(df['gyro_y'])
        mean_gz = np.mean(df['gyro_z'])
        mean_smv = np.mean(smv_list)
        std_ax = np.std(df['acc_x'])
        std_ay = np.std(df['acc_y'])
        std_az = np.std(df['acc_z'])
        std_gx = np.std(df['gyro_x'])
        std_gy = np.std(df['gyro_y'])
        std_gz = np.std(df['gyro_z'])
        std_smv = np.std(smv_list)
        std_motionless_smv = np.std(smv_list[min_idx:])
        max_smv = np.max(smv_list)
        min_smv = np.min(smv_list)
        duration = min_idx - max_idx 
        # calculate slope fature
        max_line = df.iloc[max_idx]
        min_line = df.iloc[min_idx]
        max_smv_x = max_line['acc_x']
        max_smv_y = max_line['acc_y']
        max_smv_z = max_line['acc_z']
        min_smv_x = min_line['acc_x']
        min_smv_y = min_line['acc_y']
        min_smv_z = min_line['acc_z']
        sl = slope(max_smv_x, max_smv_y, max_smv_z, min_smv_x, min_smv_y, min_smv_z)
        return {'mean_ax': mean_ax, 'mean_ay': mean_ay, 'mean_az': mean_az, 'mean_gx':mean_gx, 'mean_gy':mean_gy, 'mean_gz':mean_gz, 'smv_list':smv_list, 
                'mean_smv':mean_smv, 'std_ax':std_ax, 'std_ay':std_ay, 'std_az':std_az, 'std_gx':std_gx, 'std_gy':std_gy, 'std_gz':std_gz,
                'std_smv':std_smv, 'std_mless':std_motionless_smv,
                'max_smv':max_smv, 'min_smv':min_smv, 'slope':sl, 'duration':duration}    
    
    @staticmethod
    def getWindows(FilePath:str, MainEvent:str, SamplingRate: int, OverlapRate:float, WindowSize:float):
        window_frames = SamplingRate * WindowSize
        interval_frames = window_frames * (1 - OverlapRate)
        windows = []
        raw_data = pd.read_csv(FilePath)
        if MainEvent in ['FOL', 'FKL', 'BSC', 'SDL']:
            SMVs = np.array([])
            for line in range(0, raw_data.shape[0]):
                acc_x = raw_data['acc_x'][line]
                acc_y = raw_data['acc_y'][line]
                acc_z = raw_data['acc_z'][line]
                SMVs = np.append(SMVs, smv(acc_x, acc_y, acc_z))
            max_smv_frame = np.argmax(SMVs)
            min_smv_frame = np.argmax(SMVs)
            isFall = True
        else:
            isFall = False
        # total number of lines in the raw-data csv file
        total_frames = raw_data.shape[0]
        num_of_windows = math.ceil((total_frames - window_frames) / interval_frames) + 1
        for i in range(num_of_windows):
            start_iloc = i * interval_frames
            end_iloc = i * interval_frames + window_frames - 1
            if end_iloc > total_frames:
                end_iloc =  total_frames
            window_df = raw_data.loc[start_iloc : end_iloc, ['acc_x', 'acc_y', 'acc_z', 
                                      'gyro_x', 'gyro_y', 'gyro_z', 'label']]
            # class label : 0 for normal behaviors / 1 for falls
            if not isFall:
                window_label = 0
            else:
                if start_iloc <= max_smv_frame and min_smv_frame <= end_iloc:
                    window_label = 1 
                else:
                    window_label = 0       
            window_df = window_df.reset_index(drop=True)
            newWindow = Window(window_df, window_label)
            windows.append(newWindow)
        
        return windows

    def visualize(self):
        Xs = np.arange(0,len(self.smv_list))
        plt.plot(Xs, self.smv_list)
        plt.ylim(ymin=0, ymax=35)
        if self.label == 0:
            plt.title('Normal behavior')
        else:
            plt.title('Fall event')
        plt.xlabel('Sample Reading Number')
        plt.ylabel('Signal Matitude Vector m/$\mathregular{s^2}$')
        plt.show()
        
    def export(self):
#         if self.label == 0:
#             self.smv_list.to_csv(SavePath1)      
        newLine = pd.DataFrame([[self.mean_ax, self.mean_ay, self.mean_az, self.mean_gx, self.mean_gy, self.mean_gz, self.mean_smv,
                                 self.std_ax, self.std_ay, self.std_az, self.std_gx, self.std_gy, self.std_gz, self.std_smv, 
                                 self.std_motionless, self.max_smv, self.min_smv, 
                                 self.slope, self.duration, self.label]],
                                 columns=['mean_ax', 'mean_ay', 'mean_az', 'mean_gx', 'mean_gy', 'mean_gz', 'mean_smv', 
                                          'std_ax', 'std_ay', 'std_az', 'std_gx', 'std_gy', 'std_gz',
                                          'std_smv', 'std_mless', 'max_smv', 
                                          'min_smv', 'slope', 'duration', 'label'])
        return newLine
        

In [35]:
# Test Window class
event = 'CSI'
FilePath = os.path.join(os.getcwd(), 'MobiAct/{}/{}_1_1_annotated.csv'.format(event,event))
wins_test = Window.getWindows(FilePath, event, 200, 0.8, 5)
# wins_test[0].visualize()
# wins_test[1].visualize()
print(len(wins_test))
print(wins_test[0].export())

2
    mean_ax   mean_ay   mean_az   mean_gx   mean_gy   mean_gz  mean_smv  \
0 -1.228466 -4.478482  4.874643  0.322122 -0.206762  0.193693  9.928736   

     std_ax    std_ay    std_az    std_gx    std_gy    std_gz  std_smv  \
0  2.924666  5.370471  4.326659  0.978555  0.891077  0.866903   1.6923   

   std_mless    max_smv   min_smv      slope  duration  label  
0   1.427104  18.714399  3.281083  17.907883        13      0  


In [34]:
ADLs = {'STD':1,'WAL':1,'JOG':3,'JUM':3,'STU':6,'STN':6,'SCH':6,'SIT':1,'CHU':3,"CSI":3,'CSO':3,'LYI':0}
Falls = {'FOL':3,'FKL':3,'BSC':3,'SDL':3}
Events = ['STD','WAL','JOG','JUM','STU','STN','SCH','SIT','CHU',"CSI",'CSO','LYI','FOL','FKL','BSC','SDL']
# participants number
pNum = 67
# sampling rate (Hz)
sampleRate = 200
# overlap rate between window is 0.8 
# i.e. in the case setting windows size = 5s, the interval between windows is 1s 
overlapRate = 0.8
# window size = 5s
wSize = 5

In [38]:
dataset = pd.DataFrame(columns=['mean_ax','mean_ay','mean_az', 'mean_gx','mean_gy','mean_gz', 'mean_smv', 
                                'std_ax', 'std_ay', 'std_az', 'std_gx', 'std_gy', 'std_gz', 'std_smv', 'std_mless', 
                                'max_smv','min_smv', 'slope', 'duration', 'label'])
for event in Events:
    if event in ADLs.keys():
        trialTimes = ADLs.get(event)
    else:
        trialTimes = Falls.get(event)
    for i in range(1, pNum + 1):
        for j in range(1, trialTimes + 1):     
            fileName = 'MobiAct/{}/{}_{}_{}_annotated.csv'.format(event, event, i, j)
            dataPath = os.path.join(os.getcwd(), fileName)
            try:
                windows = Window.getWindows(dataPath, event, sampleRate, overlapRate, wSize)
                print(fileName)
                for window in windows:
                    newWindow = window.export()
                    dataset = pd.concat([dataset, newWindow], ignore_index=True)
            except FileNotFoundError:
                print('{} is not found'.format(fileName))
            continue


MobiAct/STD/STD_1_1_annotated.csv
MobiAct/STD/STD_2_1_annotated.csv
MobiAct/STD/STD_3_1_annotated.csv
MobiAct/STD/STD_4_1_annotated.csv
MobiAct/STD/STD_5_1_annotated.csv
MobiAct/STD/STD_6_1_annotated.csv
MobiAct/STD/STD_7_1_annotated.csv
MobiAct/STD/STD_8_1_annotated.csv
MobiAct/STD/STD_9_1_annotated.csv
MobiAct/STD/STD_10_1_annotated.csv
MobiAct/STD/STD_11_1_annotated.csv
MobiAct/STD/STD_12_1_annotated.csv
MobiAct/STD/STD_13_1_annotated.csv is not found
MobiAct/STD/STD_14_1_annotated.csv is not found
MobiAct/STD/STD_15_1_annotated.csv is not found
MobiAct/STD/STD_16_1_annotated.csv
MobiAct/STD/STD_17_1_annotated.csv is not found
MobiAct/STD/STD_18_1_annotated.csv
MobiAct/STD/STD_19_1_annotated.csv
MobiAct/STD/STD_20_1_annotated.csv
MobiAct/STD/STD_21_1_annotated.csv
MobiAct/STD/STD_22_1_annotated.csv
MobiAct/STD/STD_23_1_annotated.csv
MobiAct/STD/STD_24_1_annotated.csv
MobiAct/STD/STD_25_1_annotated.csv
MobiAct/STD/STD_26_1_annotated.csv
MobiAct/STD/STD_27_1_annotated.csv
MobiAct/STD/

MobiAct/JOG/JOG_32_1_annotated.csv
MobiAct/JOG/JOG_32_2_annotated.csv
MobiAct/JOG/JOG_32_3_annotated.csv
MobiAct/JOG/JOG_33_1_annotated.csv
MobiAct/JOG/JOG_33_2_annotated.csv
MobiAct/JOG/JOG_33_3_annotated.csv
MobiAct/JOG/JOG_34_1_annotated.csv
MobiAct/JOG/JOG_34_2_annotated.csv
MobiAct/JOG/JOG_34_3_annotated.csv
MobiAct/JOG/JOG_35_1_annotated.csv
MobiAct/JOG/JOG_35_2_annotated.csv
MobiAct/JOG/JOG_35_3_annotated.csv
MobiAct/JOG/JOG_36_1_annotated.csv
MobiAct/JOG/JOG_36_2_annotated.csv
MobiAct/JOG/JOG_36_3_annotated.csv
MobiAct/JOG/JOG_37_1_annotated.csv
MobiAct/JOG/JOG_37_2_annotated.csv
MobiAct/JOG/JOG_37_3_annotated.csv
MobiAct/JOG/JOG_38_1_annotated.csv
MobiAct/JOG/JOG_38_2_annotated.csv
MobiAct/JOG/JOG_38_3_annotated.csv
MobiAct/JOG/JOG_39_1_annotated.csv
MobiAct/JOG/JOG_39_2_annotated.csv
MobiAct/JOG/JOG_39_3_annotated.csv
MobiAct/JOG/JOG_40_1_annotated.csv
MobiAct/JOG/JOG_40_2_annotated.csv
MobiAct/JOG/JOG_40_3_annotated.csv
MobiAct/JOG/JOG_41_1_annotated.csv
MobiAct/JOG/JOG_41_2

MobiAct/JUM/JUM_41_2_annotated.csv
MobiAct/JUM/JUM_41_3_annotated.csv
MobiAct/JUM/JUM_42_1_annotated.csv
MobiAct/JUM/JUM_42_2_annotated.csv
MobiAct/JUM/JUM_42_3_annotated.csv
MobiAct/JUM/JUM_43_1_annotated.csv
MobiAct/JUM/JUM_43_2_annotated.csv
MobiAct/JUM/JUM_43_3_annotated.csv
MobiAct/JUM/JUM_44_1_annotated.csv
MobiAct/JUM/JUM_44_2_annotated.csv
MobiAct/JUM/JUM_44_3_annotated.csv
MobiAct/JUM/JUM_45_1_annotated.csv
MobiAct/JUM/JUM_45_2_annotated.csv
MobiAct/JUM/JUM_45_3_annotated.csv
MobiAct/JUM/JUM_46_1_annotated.csv
MobiAct/JUM/JUM_46_2_annotated.csv
MobiAct/JUM/JUM_46_3_annotated.csv
MobiAct/JUM/JUM_47_1_annotated.csv
MobiAct/JUM/JUM_47_2_annotated.csv
MobiAct/JUM/JUM_47_3_annotated.csv
MobiAct/JUM/JUM_48_1_annotated.csv
MobiAct/JUM/JUM_48_2_annotated.csv
MobiAct/JUM/JUM_48_3_annotated.csv
MobiAct/JUM/JUM_49_1_annotated.csv
MobiAct/JUM/JUM_49_2_annotated.csv
MobiAct/JUM/JUM_49_3_annotated.csv
MobiAct/JUM/JUM_50_1_annotated.csv
MobiAct/JUM/JUM_50_2_annotated.csv
MobiAct/JUM/JUM_50_3

MobiAct/STU/STU_25_4_annotated.csv
MobiAct/STU/STU_25_5_annotated.csv
MobiAct/STU/STU_25_6_annotated.csv
MobiAct/STU/STU_26_1_annotated.csv
MobiAct/STU/STU_26_2_annotated.csv
MobiAct/STU/STU_26_3_annotated.csv
MobiAct/STU/STU_26_4_annotated.csv
MobiAct/STU/STU_26_5_annotated.csv
MobiAct/STU/STU_26_6_annotated.csv
MobiAct/STU/STU_27_1_annotated.csv
MobiAct/STU/STU_27_2_annotated.csv
MobiAct/STU/STU_27_3_annotated.csv is not found
MobiAct/STU/STU_27_4_annotated.csv
MobiAct/STU/STU_27_5_annotated.csv
MobiAct/STU/STU_27_6_annotated.csv
MobiAct/STU/STU_28_1_annotated.csv
MobiAct/STU/STU_28_2_annotated.csv
MobiAct/STU/STU_28_3_annotated.csv
MobiAct/STU/STU_28_4_annotated.csv
MobiAct/STU/STU_28_5_annotated.csv
MobiAct/STU/STU_28_6_annotated.csv
MobiAct/STU/STU_29_1_annotated.csv
MobiAct/STU/STU_29_2_annotated.csv
MobiAct/STU/STU_29_3_annotated.csv
MobiAct/STU/STU_29_4_annotated.csv
MobiAct/STU/STU_29_5_annotated.csv
MobiAct/STU/STU_29_6_annotated.csv
MobiAct/STU/STU_30_1_annotated.csv is not 

MobiAct/STU/STU_64_1_annotated.csv
MobiAct/STU/STU_64_2_annotated.csv
MobiAct/STU/STU_64_3_annotated.csv
MobiAct/STU/STU_64_4_annotated.csv
MobiAct/STU/STU_64_5_annotated.csv
MobiAct/STU/STU_64_6_annotated.csv
MobiAct/STU/STU_65_1_annotated.csv
MobiAct/STU/STU_65_2_annotated.csv
MobiAct/STU/STU_65_3_annotated.csv
MobiAct/STU/STU_65_4_annotated.csv
MobiAct/STU/STU_65_5_annotated.csv
MobiAct/STU/STU_65_6_annotated.csv
MobiAct/STU/STU_66_1_annotated.csv
MobiAct/STU/STU_66_2_annotated.csv
MobiAct/STU/STU_66_3_annotated.csv
MobiAct/STU/STU_66_4_annotated.csv
MobiAct/STU/STU_66_5_annotated.csv
MobiAct/STU/STU_66_6_annotated.csv
MobiAct/STU/STU_67_1_annotated.csv
MobiAct/STU/STU_67_2_annotated.csv
MobiAct/STU/STU_67_3_annotated.csv
MobiAct/STU/STU_67_4_annotated.csv
MobiAct/STU/STU_67_5_annotated.csv
MobiAct/STU/STU_67_6_annotated.csv
MobiAct/STN/STN_1_1_annotated.csv
MobiAct/STN/STN_1_2_annotated.csv
MobiAct/STN/STN_1_3_annotated.csv
MobiAct/STN/STN_1_4_annotated.csv
MobiAct/STN/STN_1_5_anno

...

MobiAct/CSO/CSO_7_1_annotated.csv
MobiAct/CSO/CSO_7_2_annotated.csv
MobiAct/CSO/CSO_7_3_annotated.csv
MobiAct/CSO/CSO_8_1_annotated.csv
MobiAct/CSO/CSO_8_2_annotated.csv
MobiAct/CSO/CSO_8_3_annotated.csv
MobiAct/CSO/CSO_9_1_annotated.csv
MobiAct/CSO/CSO_9_2_annotated.csv
MobiAct/CSO/CSO_9_3_annotated.csv
MobiAct/CSO/CSO_10_1_annotated.csv
MobiAct/CSO/CSO_10_2_annotated.csv
MobiAct/CSO/CSO_10_3_annotated.csv
MobiAct/CSO/CSO_11_1_annotated.csv
MobiAct/CSO/CSO_11_2_annotated.csv
MobiAct/CSO/CSO_11_3_annotated.csv
MobiAct/CSO/CSO_12_1_annotated.csv
MobiAct/CSO/CSO_12_2_annotated.csv
MobiAct/CSO/CSO_12_3_annotated.csv
MobiAct/CSO/CSO_13_1_annotated.csv is not found
MobiAct/CSO/CSO_13_2_annotated.csv is not found
MobiAct/CSO/CSO_13_3_annotated.csv is not found
MobiAct/CSO/CSO_14_1_annotated.csv is not found
MobiAct/CSO/CSO_14_2_annotated.csv is not found
MobiAct/CSO/CSO_14_3_annotated.csv is not found
MobiAct/CSO/CSO_15_1_annotated.csv is not found
MobiAct/CSO/CSO_15_2_annotated.csv is not f

MobiAct/FOL/FOL_16_2_annotated.csv
MobiAct/FOL/FOL_16_3_annotated.csv
MobiAct/FOL/FOL_17_1_annotated.csv
MobiAct/FOL/FOL_17_2_annotated.csv
MobiAct/FOL/FOL_17_3_annotated.csv
MobiAct/FOL/FOL_18_1_annotated.csv
MobiAct/FOL/FOL_18_2_annotated.csv
MobiAct/FOL/FOL_18_3_annotated.csv
MobiAct/FOL/FOL_19_1_annotated.csv
MobiAct/FOL/FOL_19_2_annotated.csv
MobiAct/FOL/FOL_19_3_annotated.csv
MobiAct/FOL/FOL_20_1_annotated.csv
MobiAct/FOL/FOL_20_2_annotated.csv
MobiAct/FOL/FOL_20_3_annotated.csv
MobiAct/FOL/FOL_21_1_annotated.csv
MobiAct/FOL/FOL_21_2_annotated.csv
MobiAct/FOL/FOL_21_3_annotated.csv
MobiAct/FOL/FOL_22_1_annotated.csv
MobiAct/FOL/FOL_22_2_annotated.csv
MobiAct/FOL/FOL_22_3_annotated.csv
MobiAct/FOL/FOL_23_1_annotated.csv
MobiAct/FOL/FOL_23_2_annotated.csv
MobiAct/FOL/FOL_23_3_annotated.csv
MobiAct/FOL/FOL_24_1_annotated.csv is not found
MobiAct/FOL/FOL_24_2_annotated.csv is not found
MobiAct/FOL/FOL_24_3_annotated.csv is not found
MobiAct/FOL/FOL_25_1_annotated.csv
MobiAct/FOL/FOL_

MobiAct/FKL/FKL_26_3_annotated.csv
MobiAct/FKL/FKL_27_1_annotated.csv
MobiAct/FKL/FKL_27_2_annotated.csv
MobiAct/FKL/FKL_27_3_annotated.csv
MobiAct/FKL/FKL_28_1_annotated.csv
MobiAct/FKL/FKL_28_2_annotated.csv
MobiAct/FKL/FKL_28_3_annotated.csv
MobiAct/FKL/FKL_29_1_annotated.csv
MobiAct/FKL/FKL_29_2_annotated.csv
MobiAct/FKL/FKL_29_3_annotated.csv
MobiAct/FKL/FKL_30_1_annotated.csv
MobiAct/FKL/FKL_30_2_annotated.csv
MobiAct/FKL/FKL_30_3_annotated.csv
MobiAct/FKL/FKL_31_1_annotated.csv
MobiAct/FKL/FKL_31_2_annotated.csv
MobiAct/FKL/FKL_31_3_annotated.csv
MobiAct/FKL/FKL_32_1_annotated.csv
MobiAct/FKL/FKL_32_2_annotated.csv
MobiAct/FKL/FKL_32_3_annotated.csv
MobiAct/FKL/FKL_33_1_annotated.csv
MobiAct/FKL/FKL_33_2_annotated.csv
MobiAct/FKL/FKL_33_3_annotated.csv
MobiAct/FKL/FKL_34_1_annotated.csv
MobiAct/FKL/FKL_34_2_annotated.csv
MobiAct/FKL/FKL_34_3_annotated.csv
MobiAct/FKL/FKL_35_1_annotated.csv
MobiAct/FKL/FKL_35_2_annotated.csv
MobiAct/FKL/FKL_35_3_annotated.csv
MobiAct/FKL/FKL_36_1

MobiAct/BSC/BSC_37_2_annotated.csv
MobiAct/BSC/BSC_37_3_annotated.csv
MobiAct/BSC/BSC_38_1_annotated.csv
MobiAct/BSC/BSC_38_2_annotated.csv
MobiAct/BSC/BSC_38_3_annotated.csv
MobiAct/BSC/BSC_39_1_annotated.csv
MobiAct/BSC/BSC_39_2_annotated.csv
MobiAct/BSC/BSC_39_3_annotated.csv
MobiAct/BSC/BSC_40_1_annotated.csv
MobiAct/BSC/BSC_40_2_annotated.csv
MobiAct/BSC/BSC_40_3_annotated.csv
MobiAct/BSC/BSC_41_1_annotated.csv is not found
MobiAct/BSC/BSC_41_2_annotated.csv is not found
MobiAct/BSC/BSC_41_3_annotated.csv is not found
MobiAct/BSC/BSC_42_1_annotated.csv
MobiAct/BSC/BSC_42_2_annotated.csv
MobiAct/BSC/BSC_42_3_annotated.csv
MobiAct/BSC/BSC_43_1_annotated.csv
MobiAct/BSC/BSC_43_2_annotated.csv
MobiAct/BSC/BSC_43_3_annotated.csv
MobiAct/BSC/BSC_44_1_annotated.csv
MobiAct/BSC/BSC_44_2_annotated.csv
MobiAct/BSC/BSC_44_3_annotated.csv
MobiAct/BSC/BSC_45_1_annotated.csv
MobiAct/BSC/BSC_45_2_annotated.csv
MobiAct/BSC/BSC_45_3_annotated.csv
MobiAct/BSC/BSC_46_1_annotated.csv
MobiAct/BSC/BSC_

MobiAct/SDL/SDL_47_3_annotated.csv
MobiAct/SDL/SDL_48_1_annotated.csv
MobiAct/SDL/SDL_48_2_annotated.csv
MobiAct/SDL/SDL_48_3_annotated.csv
MobiAct/SDL/SDL_49_1_annotated.csv
MobiAct/SDL/SDL_49_2_annotated.csv
MobiAct/SDL/SDL_49_3_annotated.csv
MobiAct/SDL/SDL_50_1_annotated.csv is not found
MobiAct/SDL/SDL_50_2_annotated.csv is not found
MobiAct/SDL/SDL_50_3_annotated.csv is not found
MobiAct/SDL/SDL_51_1_annotated.csv
MobiAct/SDL/SDL_51_2_annotated.csv
MobiAct/SDL/SDL_51_3_annotated.csv
MobiAct/SDL/SDL_52_1_annotated.csv
MobiAct/SDL/SDL_52_2_annotated.csv
MobiAct/SDL/SDL_52_3_annotated.csv
MobiAct/SDL/SDL_53_1_annotated.csv
MobiAct/SDL/SDL_53_2_annotated.csv
MobiAct/SDL/SDL_53_3_annotated.csv
MobiAct/SDL/SDL_54_1_annotated.csv
MobiAct/SDL/SDL_54_2_annotated.csv
MobiAct/SDL/SDL_54_3_annotated.csv
MobiAct/SDL/SDL_55_1_annotated.csv
MobiAct/SDL/SDL_55_2_annotated.csv
MobiAct/SDL/SDL_55_3_annotated.csv
MobiAct/SDL/SDL_56_1_annotated.csv
MobiAct/SDL/SDL_56_2_annotated.csv
MobiAct/SDL/SDL_

In [40]:
SavePath = os.path.join(os.getcwd(), 'Windows_Data/dataset_19f.csv')
dataset = dataset.dropna(axis=0, how='any')
# dataset = dataset[dataset['max_smv'] > 16.5]
# dataset.to_csv(SavePath,index=None)
d = dataset.loc[dataset['max_smv']>16.5]
d.to_csv(SavePath,index=None)

In [41]:
d.loc[dataset['label'] == 1].describe()

,mean_ax,mean_ay,mean_az,mean_gx,mean_gy,mean_gz,mean_smv,std_ax,std_ay,std_az,std_gx,std_gy,std_gz,std_smv,std_mless,max_smv,min_smv,slope
count,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000
mean,0.225473,1.997096,2.099988,-0.095524,0.008988,0.006039,10.033300,3.116171,3.782372,3.696545,0.847559,0.743248,0.626816,2.375672,1.849458,25.652472,3.059850,24.639264
std,4.749534,2.880429,4.985206,0.214385,0.110224,0.136209,0.214833,1.048113,1.216842,1.473344,0.243368,0.323045,0.216797,0.445436,0.769948,2.796419,1.475822,3.455643
min,-9.416236,-7.579210,-9.868187,-0.482011,-0.368216,-0.316814,9.339711,0.639318,0.477201,0.810486,0.115565,0.120404,0.094861,1.014196,0.229472,16.886193,0.302170,11.121601
25%,-3.944016,0.388411,-2.899911,-0.272365,-0.048797,-0.114414,9.881906,2.318795,2.902645,2.479693,0.680471,0.509575,0.484500,2.046723,1.238873,23.719808,1.890238,22.285522
50%,0.027226,2.583475,3.998813,-0.183321,0.009851,-0.011506,10.022019,3.094177,3.701869,3.537816,0.838895,0.669447,0.605401,2.382051,1.776720,25.769089,2.986808,24.702643
75%,4.532634,4.174693,5.834172,0.110921,0.062112,0.132277,10.178270,3.817947,4.579014,4.856561,1.000517,0.906467,0.744426,2.706874,2.412042,27.555061,4.046217,27.039175
max,9.460887,7.711178,9.175232,0.457516,0.446086,0.344791,10.828897,6.516456,7.807418,7.567937,1.762219,2.020070,1.556490,3.856655,4.650434,32.885551,7.907462,34.798956


In [42]:
d.loc[dataset['label'] == 0].describe()

,mean_ax,mean_ay,mean_az,mean_gx,mean_gy,mean_gz,mean_smv,std_ax,std_ay,std_az,std_gx,std_gy,std_gz,std_smv,std_mless,max_smv,min_smv,slope
count,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000,32445.000000
mean,0.157974,6.809539,0.299012,-0.017841,0.000069,0.014944,10.533101,2.888318,4.543920,3.097431,1.355008,1.326609,0.830542,4.404119,4.357247,22.412787,2.803009,20.805016
std,2.584707,5.825318,1.361481,0.087278,0.169509,0.088578,0.928933,1.210713,2.136668,0.966697,0.483178,0.565673,0.378146,1.815329,1.929006,2.768952,1.799965,4.069531
min,-10.208453,-10.271453,-10.005843,-0.632424,-1.094711,-0.470894,7.068501,0.280671,0.280595,0.258406,0.057227,0.059248,0.039491,0.485820,0.000000,16.500085,0.030987,8.746319
25%,-0.937115,7.398452,-0.371213,-0.062035,-0.059649,-0.038805,10.148719,2.050326,2.963200,2.452342,1.019418,0.959005,0.559485,3.166115,3.047411,20.391301,1.011662,17.822137
50%,0.125342,9.445239,0.228169,-0.016986,-0.003400,0.012889,10.474581,2.664403,3.641658,2.963221,1.296426,1.233842,0.763055,3.819427,3.853015,22.330187,2.995103,21.124180
75%,1.409760,9.790581,0.905959,0.030023,0.049957,0.067939,10.834604,3.390182,6.754279,3.592004,1.635481,1.536175,1.020015,5.744368,5.770897,24.298160,4.152255,23.815661
max,10.119549,10.565246,9.462464,0.619892,0.949169,0.468317,15.180757,9.127654,10.679623,8.141915,3.607131,4.818761,2.835267,10.917130,11.371032,32.430852,9.237574,32.491868


In [39]:
dataset

,mean_ax,mean_ay,mean_az,mean_gx,mean_gy,mean_gz,mean_smv,std_ax,std_ay,std_az,std_gx,std_gy,std_gz,std_smv,std_mless,max_smv,min_smv,slope,duration,label
0,0.323985,-9.738408,-0.670010,-0.045889,0.010005,0.023417,9.771294,0.206927,0.023299,0.211992,0.027749,0.052291,0.021290,0.024164,0.021095,9.834680,9.693022,0.394635,526,0
1,0.282628,-9.740570,-0.709322,-0.042205,0.015858,0.018571,9.773626,0.177241,0.020855,0.175231,0.025708,0.050529,0.018339,0.022301,0.018656,9.834680,9.693022,0.394635,526,0
2,0.323013,-9.741705,-0.676915,-0.037399,0.017715,0.015774,9.772474,0.133484,0.019510,0.141746,0.022021,0.047497,0.015141,0.020293,0.017222,9.832309,9.693022,0.484351,429,0
3,0.329039,-9.743705,-0.682181,-0.045086,0.004811,0.021423,9.774087,0.093642,0.018023,0.103021,0.009763,0.019063,0.007067,0.018475,0.017242,9.813454,9.693022,0.179540,-698,0
4,0.292710,-9.743753,-0.724106,-0.043660,0.007157,0.020495,9.775211,0.036112,0.017630,0.052083,0.008794,0.019570,0.005952,0.018004,0.017046,9.813841,9.693022,0.157106,-863,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56912,-6.419022,0.573473,5.485222,-0.070136,-0.117606,-0.257927,10.021342,2.844166,3.495058,3.713393,0.680209,0.582162,0.747469,2.299367,0.733250,30.279845,4.210766,31.460945,52,1
56913,-6.756292,-0.492942,6.750617,0.023207,0.026931,0.026819,9.708413,0.832306,1.433013,0.670122,0.235465,0.132096,0.225558,0.635101,0.635101,15.671375,5.639311,11.911746,-11,0
56914,-6.924817,-0.191492,6.675758,-0.044197,0.017337,0.009419,9.623634,0.137662,0.189339,0.115696,0.037994,0.037674,0.023837,0.096780,0.090565,9.926912,9.240420,0.743846,31,0
56915,-6.942873,-0.237854,6.659097,-0.039978,0.009449,0.013769,9.624513,0.108096,0.123507,0.085514,0.028833,0.021968,0.011027,0.081397,0.073068,9.911831,9.358716,0.577394,-10,0
